# Pipline

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import glob

tangle_data = pd.read_excel(
    "OMSK-PKN_GP1.1_NEW.xlsx",
    sheet_name="Фасады_Секции по файлу",
    skiprows=7,
    header=0 
)

#=== Clean tangle data
tangle_data = tangle_data[['Утвержденный номер','Справочник','Номер позиции', 'Id']]
tangle_data = tangle_data[tangle_data['Справочник'] == '2. Работы строительно-монтажные NEW']

tangle_data['section'] = tangle_data['Утвержденный номер'].str.split('.').str[0:2].str.join('.')
# tangle_data = tangle_data[tangle_data['section'] != tangle_data['Номер позиции']]
section_files = {
    "1.1":'revit_params_OMSK-PKN_GP1.1_01_AR.csv',
    "1.2":'revit_params_OMSK-PKN_GP1.1_02,08_AR.csv',
    "1.3":'revit_params_OMSK-PKN_GP1.1_03,05,07_AR.csv',
    "1.4":'revit_params_OMSK-PKN_GP1.1_04_AR.csv',
    "1.5":'revit_params_OMSK-PKN_GP1.1_03,05,07_AR.csv',
    "1.6":'revit_params_OMSK-PKN_GP1.1_06_AR.csv',
    "1.7":'revit_params_OMSK-PKN_GP1.1_03,05,07_AR.csv',
    "1.8":'revit_params_OMSK-PKN_GP1.1_02,08_AR.csv',
    "1.9":'revit_params_OMSK-PKN_GP1.1_09_AR.csv',
}
tangle_data['section'] = tangle_data['section'].map(section_files)

tangle_data.dropna(inplace=True)
tangle_data.drop(columns=['Утвержденный номер', 'Справочник'], inplace=True)

tangle_data.rename(columns={'Номер позиции': "Position", "Id":"ElementId"}, inplace=True)
tangle_data['ElementId'] = tangle_data['ElementId'].astype('int64')
id_groups = tangle_data.groupby('ElementId')['Position'].apply(list).reset_index()

#==== Concat all csv
csv_files = glob.glob('*.csv')
revit_data = pd.DataFrame()
for file in csv_files:
    if "revit_params_OM" in file:
        tangle_data_slice = tangle_data[tangle_data['section'] == file]
        df = pd.read_csv(file)

        marged_df = tangle_data_slice.merge(df, left_on='ElementId', right_on='ElementId')
        revit_data = pd.concat([revit_data, marged_df], sort=False, axis=0)
del revit_data['section']

#=== marged_df split and clear
target_cols = ['ElementId', 'Position']
X_train = revit_data.drop(columns=target_cols, axis=1)
X_df_to_drop = ['Марка', 'Граница помещения','Комментарии','Линия привязки','Маркировка типоразмера','ADSK_Этаж', 'Цвет штриховки при низкой детализации', 'ADSK_Номер секции','ADSK_Группирование']
X_train.drop(columns=X_df_to_drop, inplace=True)
X_df_to_drop2 =  X_train.columns[X_train.columns.str.contains('PS_Секция')]
X_train.drop(columns=X_df_to_drop2, inplace=True)

X_train['Зависимость снизу'] = X_train['Зависимость снизу'].str.split('(', expand=True)[0]
X_train['Зависимость сверху'] = X_train['Зависимость сверху'].str.split('(', expand=True)[0]
X_train['Уровень'] = X_train['Уровень'].str.split('(', expand=True)[0]

#=== Matrix of targets 
y_train = revit_data[target_cols]
y_train = pd.get_dummies(y_train, columns=['Position'])
del y_train['ElementId']
y_train = y_train.astype(np.int8)
y_train.reset_index(inplace=True, drop=True)
constant_cols = X_train.columns[X_train.nunique() <= 1]
X_train.drop(columns=constant_cols, inplace=True)
X_train.reset_index(inplace=True, drop=True)
X_train

C:\Users\homer\AppData\Local\Temp\ipykernel_15576\2200594632.py:47: DtypeWarning: Columns (58,83,100,101,106,112,116,119,121,123,124,142,143,149,150,152,156,157,164,167,170,175,177,183,185,186,187,188,189,190,194,196,198,204,207,211,212,215,231,238,239,249,253,286,318,319,323,331,332,333,339,344,345,346,347,348,403,407,408,409,410,432,448,450,480,483,488,489,490,491,492,493,494,495,498,499,500,572,591,592,595,596,600,601,665,673,675,676,678,680,696,697,698,699,700,701,702,703,704,705,706,707,711,712,713,714,715,716,726,727,730,731,732,733,734,735,736,740,741,742,743,744,745,746,747,749,751,759,760,762,768,777,778,779,780,782,809,810,833,837,858,868,869,870,871,872,875,876,887,888,889,890,891,892,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,958,973,974,975,976,983,993,994,995,996,1010,1013,1014,1015,1019,1028,1030,1031,1032,1033,1040,1041,1042,1043,1044,1045

,#_Высота отделки,#_Ширина откоса,ADSK_Зона,ADSK_Материал,ADSK_Основной материал элемента,ADSK_Примечание,ADSK_Размер_Высота,ADSK_Размер_Длина,ADSK_Размер_Площадь,ADSK_Размер_Толщина,...,яНахлестОтделкиНиз,яОткосВерхний,яОткосЛевый,яОткосПравый,яУдлинениеОткосаЛевого(вверх),яУдлинениеОткосаПравого(вверх),Отметка низа сердцевины,Отметка низа съемки,Отметка сердцевины верха,Отметка съемки верха
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.492126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.328084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.492126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.492126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.492126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
Векторизация категориальных переменных

In [5]:
import re
import unidecode
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def _clean_text(text: str) -> str:
    if not isinstance(text, str) or text.strip() == "":
        return ""
    
    # text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'(\d+)-[йя]', r'\1', text)
    text = re.sub(r'\s+', ' ', text)
    # text = unidecode.unidecode(text)
    text = text.lower().strip()
    
    return text

def _lemmatize_text(text: str) -> str:
    if not text:
        return ""
    tokens = text.split()
    lemmatized = []
    for token in tokens:
        parsed = morph.parse(token)
        lemma = parsed[0].normal_form

        lemmatized.append(lemma)
    return " ".join(lemmatized)

import re

cats = X_train.select_dtypes('object').columns.to_list()

for col in cats:
    X_train[col] = X_train[col].apply(_clean_text)#.apply(_lemmatize_text)
X_train.replace("", np.nan, inplace=True)

---

1. нет части праметров, например для стен Длина, ШИрина, обьем. Может из-за нелдостатка параметров не клеется обучение

In [6]:
for col in y_train.columns:
    count = y_train[col].value_counts()
    print(f'{col}: 0-{count[0]}, 1-{count[1]}')

Position_004.014.033.001.002: 0-20321, 1-1751
Position_004.014.033.001.005: 0-19748, 1-2324
Position_004.014.033.001.008: 0-19748, 1-2324
Position_004.014.033.001.012: 0-19748, 1-2324
Position_004.014.033.001.013: 0-19748, 1-2324
Position_004.014.033.002.001: 0-20327, 1-1745
Position_004.014.033.002.002: 0-20327, 1-1745
Position_004.014.033.002.003: 0-20327, 1-1745
Position_004.014.033.002.004: 0-20327, 1-1745
Position_004.014.033.002.005: 0-20327, 1-1745
Position_004.014.033.004.001: 0-21768, 1-304
Position_004.014.033.004.002: 0-21324, 1-748
Position_004.014.033.004.003: 0-21768, 1-304
Position_004.014.033.004.004: 0-21768, 1-304
Position_004.014.033.004.005: 0-21768, 1-304
Position_004.014.033.004.006: 0-21768, 1-304
Position_004.014.033.006.001.1: 0-22040, 1-32


```Position_004.014.033.001.005: 0-19748, 1-2324
Position_004.014.033.001.008: 0-19748, 1-2324
Position_004.014.033.001.012: 0-19748, 1-2324
Position_004.014.033.001.013: 0-19748, 1-2324
Position_004.014.033.002.001: 0-20327, 1-1745
Position_004.014.033.002.002: 0-20327, 1-1745
Position_004.014.033.002.003: 0-20327, 1-1745
Position_004.014.033.002.004: 0-20327, 1-1745
Position_004.014.033.002.005: 0-20327, 1-1745

На этих таргетах обучение идет еле еле. Надо понять почему. 


1. думаю стоит на ADSK_зона накинуть метку. Самое значение не важно

In [117]:
#=== TESTING
target_cols = ['ElementId']
X_train = revit_data.drop(columns=target_cols, axis=1)
X_df_to_drop = ['Марка', 'Граница помещения','Комментарии','Линия привязки','Маркировка типоразмера','ADSK_Этаж', 'Цвет штриховки при низкой детализации', 'Код типа','ADSK_Номер секции','ADSK_Группирование']
X_train.drop(columns=X_df_to_drop, inplace=True)
X_df_to_drop2 =  X_train.columns[X_train.columns.str.contains('PS_Секция')]
X_train.drop(columns=X_df_to_drop2, inplace=True)

X_train['Зависимость снизу'] = X_train['Зависимость снизу'].str.split('(', expand=True)[0]
X_train['Зависимость сверху'] = X_train['Зависимость сверху'].str.split('(', expand=True)[0]
X_train['Уровень'] = X_train['Уровень'].str.split('(', expand=True)[0]

#=== Matrix of targets 
target_cols = ['ElementId', 'Position']
y_train = revit_data[target_cols]
y_train = pd.get_dummies(y_train, columns=['Position'])
del y_train['ElementId']
y_train = y_train.astype(np.int8)
y_train.reset_index(drop=True, inplace=True)

constant_cols = X_train.columns[X_train.nunique() <= 1]
X_train.drop(columns=constant_cols, inplace=True)
X_train.reset_index(drop=True, inplace=True)
X_train
# first = X_train[X_train['Position'] == '004.014.033.001.002']
# fd = first.columns[first.nunique() <= 1]
# first = first.drop(columns=fd) 

# second = X_train[X_train['Position'] == '004.014.033.001.005']

,Position,#_Высота отделки,#_Ширина откоса,ADSK_Зона,ADSK_Материал,ADSK_Основной материал элемента,ADSK_Примечание,ADSK_Размер_Высота,ADSK_Размер_Длина,ADSK_Размер_Площадь,...,яНахлестОтделкиНиз,яОткосВерхний,яОткосЛевый,яОткосПравый,яУдлинениеОткосаЛевого(вверх),яУдлинениеОткосаПравого(вверх),Отметка низа сердцевины,Отметка низа съемки,Отметка сердцевины верха,Отметка съемки верха
0,004.014.033.001.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,004.014.033.001.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,004.014.033.001.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,004.014.033.001.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,004.014.033.001.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22067,004.014.033.006.001.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22068,004.014.033.006.001.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22069,004.014.033.006.001.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22070,004.014.033.006.001.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


```H0: Признаки не значимы. H1: признаки значимы (p<0,5) ```
Те параметры, что будут в словаре - являются статистически значимыми

In [189]:
X_train['Материал откоса нижний'].value_counts()

Материал откоса нижний
PS_Штукатурка_Tikkurila F485    434
PS_Штукатурка_Tikkurila J498    189
PS_Штукатурка_Tikkurila M498     90
PS_Штукатурка_Tikkurila X498     73
PS_Штукатурка_Tikkurila H485     44
Name: count, dtype: int64

In [7]:
from scipy.stats import chi2_contingency

stat_params = {}
for y_col in y_train.columns:
    params = []

    for col in X_train.columns:
        contingency_table = pd.crosstab(X_train[col], y_train[y_col])
        chi2, p, dof, exp = chi2_contingency(contingency_table)

        if p < 0.05:
            params.append(col)

    stat_params[y_col] = params

stat_params


{'Position_004.014.033.001.002': ['ADSK_Размер_Площадь',
  'ADSK_Размер_Толщина',
  'Category',
  'Включить аналитическую модель',
  'Высота',
  'ВысотаОкна',
  'Длина',
  'Зависимость сверху',
  'Зависимость снизу',
  'Категория',
  'Код типа',
  'Коэффициент теплопередачи (U)',
  'Материал несущих конструкций',
  'Неприсоединенная высота',
  'Несущие конструкции',
  'Объем',
  'Огибание в местах вставки элементов',
  'Огибание в торцах стен',
  'Описание',
  'Основа',
  'Отметка верха',
  'Отметка низа',
  'Периметр',
  'Площадь',
  'Рабочий набор',
  'Семейство',
  'Семейство и типоразмер',
  'Смещение от главной модели',
  'Смещение сверху',
  'Смещение снизу',
  'Стадия возведения',
  'Стадия сноса',
  'Тепловая нагрузка',
  'Термостойкость',
  'Тип',
  'Толщина',
  'Толщина утеплителя',
  'Уровень',
  'Функция',
  'Ширина',
  'ШиринаОкна',
  'Штриховка при низкой детализации'],
 'Position_004.014.033.001.005': ['ADSK_Материал',
  'ADSK_Размер_Длина',
  'ADSK_Размер_Площадь',
  'A

## Обучение

In [178]:
X_train

,#_Высота отделки,#_Ширина откоса,ADSK_Зона,ADSK_Материал,ADSK_Основной материал элемента,ADSK_Примечание,ADSK_Размер_Высота,ADSK_Размер_Длина,ADSK_Размер_Площадь,ADSK_Размер_Толщина,...,яНахлестОтделкиНиз,яОткосВерхний,яОткосЛевый,яОткосПравый,яУдлинениеОткосаЛевого(вверх),яУдлинениеОткосаПравого(вверх),Отметка низа сердцевины,Отметка низа съемки,Отметка сердцевины верха,Отметка съемки верха
0,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,0.492126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,0.328084,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,0.492126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,0.492126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,0.492126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22067,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,0.032808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22068,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,0.032808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22069,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,0.032808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22070,NaN,NaN,nan,nan,nan,nan,NaN,NaN,NaN,0.032808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


STk-fold

In [ ]:
# catb_model = CatBoostClassifier(
#     random_state=42,
#     nan_mode='Max',           # или 'Min' тут хз
#     max_depth=4,              
#     learning_rate=1,        # мб стоит изсенить 
#     n_estimators=500,         # количество деревьев пока 200
#     verbose=100,              # лог каждые 100 итераций, иначе иного инфы
#     scale_pos_weight=None,    # комит, так как не задаю class_weights
#     cat_features=cat_features,# база
#     early_stopping_rounds=15,
#     eval_metric='F1',         # попробую задалансить f1
#     auto_class_weights='Balanced',  # !автомитическая балансировака классов. Имба
# )

# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# global_f1_score = []
# multi_catb = []
# for i in range(y_train.shape[1]): # 2 в одном, и проверяю устойчивость модели и обучаю сразу. 
#     y_i = y_train.iloc[:, i]

#     f1= 0
#     count = 0
#     for fold, (train, valid) in enumerate(kf.split(X_train, y_i)):
#         print(f"\n--- Fold{i}_{fold + 1} ---")

#         X_train_fold = X_train.iloc[train]
#         X_val_fold = X_train.iloc[valid]
#         y_train_fold = y_i.iloc[train]
#         y_val_fold = y_i.iloc[valid]
        
#         #---Копия модели для каждого фолда
#         model = CatBoostClassifier(**catb_model.get_params())
        
#         #---Обучение
#         model.fit(
#             X_train_fold, y_train_fold,
#             eval_set=(X_val_fold, y_val_fold),
#             # plot=False  # включает графики. Прикольно
#         )
        
#         #---Предсказания
#         y_pred = model.predict(X_val_fold)
#         f1 += f1_score(y_val_fold, y_pred, average='macro')
#         count += 1

#     f1_mean_score = f1/count

#     print(f"\n-----------F1 на валидации: {(f1_mean_score):.4f}\n")
#     # global_f1_score.append(f1_mean_score)

#     # cbc = CatBoostClassifier(**catb_model.get_params())
#     # cbc.fit(X_train, y_i)

#     # multi_catb.append(cbc)

In [8]:
cat_features = []
for col in X_train.select_dtypes('object').columns:
    X_train[col] = X_train[col].astype(str)
    cat_features.append(col)
    X_train[col] = X_train[col].astype('category')

In [9]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
import joblib

import json

# CatBoost - Ручное обучение через цикл. На каждый таргет своя модель. + баланс классов
'''
Когда включил график, то заметил что у меня часть моделей ток то быстро обучется, и остальные 100 условных операция идет просто переобучение 
Как я понял есть 2 варианта с этим бороться. Тупо поставить
'''
feature_weights = np.ones(len(X_train.columns))
feature_weights[0] = 0.5
feature_weights
multi_catb = []
catb_model = CatBoostClassifier(
    nan_mode='Max',           # или 'Min' тут хз
    max_depth=5,              
    learning_rate=0.1,        # мб стоит изсенить 
    n_estimators=600,         # количество деревьев пока 200
    l2_leaf_reg=5,
    verbose=300,              # лог каждые 100 итераций, иначе иного инфы
    scale_pos_weight=None,    # комит, так как не задаю class_weights
    cat_features=cat_features,# база
    early_stopping_rounds=50,
    # eval_metric='F1',         # попробую задалансить f1
    auto_class_weights='Balanced',  # !автомитическая балансировака классов. Имба
)

# for i in range(y_train.shape[1]):
# y_i = y_train.iloc[:, i]
y_i = y_train
# classes = np.unique(y_i)
# class_weights = compute_class_weight('balanced', classes=classes, y=y_i)
# class_weight_dict = dict(zip(classes, class_weights))
X = X_train
# X = X_train[stat_params[y_train.columns[i]]]
# cat_features = X.select_dtypes('category').columns.to_list()

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, 
    y_i, 
    test_size=0.33, 
    random_state=42, 
    stratify=y_i
)

# catb_model = CatBoostClassifier(
#     # task_type="GPU",
#     # devices='0',
#     depth=6,
#     learning_rate=0.1,
#     iterations = 1500,
#     verbose=100,
#     eval_metric='F1',
#     early_stopping_rounds=50,
#     class_weights=class_weight_dict,
#     # auto_class_weights='Balanced',
#     cat_features=cat_features
# )

cbc = CatBoostClassifier(**catb_model.get_params())
cbc.fit(X_train_split, y_train_split, plot=True, eval_set=(X_val, y_val))

multi_catb.append(cbc)

joblib.dump(multi_catb, 'catb.pkl')



TypeError: unhashable type: 'numpy.ndarray'

In [8]:
import joblib
import json

# Сохранение моделей, cat_features и фичей
# joblib.dump(multi_lgbm, 'lgbm.pkl')
joblib.dump(multi_catb, 'catb.pkl')

with open('features.json', 'w') as f:
    json.dump(X_train.columns.to_list(), f)

print("Модели и метаданные сохранены.")

Модели и метаданные сохранены.


In [9]:
with open('trained_targets.json', 'w') as f:
    json.dump(y_train.columns.to_list(), f)

In [10]:
with open('cat_features.json', 'w') as f:
    json.dump(cat_features, f)

In [11]:
with open('features.json', 'r') as f:
    features = json.load(f)


In [12]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score

#=== Предсказание вероятностей
# y_pred_proba_lgbm = np.array([clf.predict_proba(X_df)[:, 1] for clf in multi_lgbm.estimators_]).T
y_pred_proba_ensemble = np.array([clf.predict_proba(X_train)[:, 1] for clf in multi_catb]).T

# y_pred_proba_ensemble = (y_pred_proba_lgbm + y_pred_proba_catb) / 2

#=== отсечения порогом вероятности 
y_pred_ensemble = (y_pred_proba_ensemble > 0.5).astype(int)

In [14]:
y_train

,Position_004.014.033.001.002,Position_004.014.033.001.005,Position_004.014.033.001.008,Position_004.014.033.001.012,Position_004.014.033.001.013,Position_004.014.033.002.001,Position_004.014.033.002.002,Position_004.014.033.002.003,Position_004.014.033.002.004,Position_004.014.033.002.005,Position_004.014.033.004.001,Position_004.014.033.004.002,Position_004.014.033.004.003,Position_004.014.033.004.004,Position_004.014.033.004.005,Position_004.014.033.004.006,Position_004.014.033.006.001.1
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3009,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3010,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3011,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [13]:
y_pred_ensemble

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 1, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 1]])

In [15]:
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score

#=== Точность по объектам (все метки совпали)
exact_match = accuracy_score(y_train, y_pred_ensemble, normalize=True)

#=== Hamming Accuracy (доля правильно предсказанных меток)
hamming_acc = 1 - hamming_loss(y_train, y_pred_ensemble)

#=== F1, Precision, Recall — с разными типами усреднения
f1_macro = f1_score(y_train, y_pred_ensemble, average='macro')
f1_micro = f1_score(y_train, y_pred_ensemble, average='micro')

precision_macro = precision_score(y_train, y_pred_ensemble, average='macro', zero_division=0)
precision_micro = precision_score(y_train, y_pred_ensemble, average='micro', zero_division=0)

recall_macro = recall_score(y_train, y_pred_ensemble, average='macro', zero_division=0)
recall_micro = recall_score(y_train, y_pred_ensemble, average='micro', zero_division=0)

#=== Метрики
print("Результаты ансамбля LGBM + CatBoost:")
print(f"Macro F1 Score:       {f1_macro:.4f}")
print(f"Micro F1 Score:       {f1_micro:.4f}")
print(f"Macro Precision:      {precision_macro:.4f}")
print(f"Micro Precision:      {precision_micro:.4f}")
print(f"Macro Recall:         {recall_macro:.4f}")
print(f"Micro Recall:         {recall_micro:.4f}")

Результаты ансамбля LGBM + CatBoost:
Macro F1 Score:       0.3681
Micro F1 Score:       0.3603
Macro Precision:      0.2580
Micro Precision:      0.2262
Macro Recall:         0.8285
Micro Recall:         0.8844


# Проверка на тесовых данных

Интерпретация результатов 

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.preprocessing import MultiLabelBinarizer

test_data = pd.read_excel("KRGN-MK_GP07_Фасады_NEW.xlsx", sheet_name="Фасады_Секции по файлу")

#=== Clean data
test_tangle_data = test_data[['Unnamed: 2', 'Unnamed: 3']]
test_tangle_data = test_tangle_data.dropna()

test_tangle_data = test_tangle_data[test_tangle_data['Unnamed: 2'].str.count(r"\.") > 1]

test_tangle_data.rename(columns={'Unnamed: 2': "Position", "Unnamed: 3":"ElementId"}, inplace=True)
test_tangle_data = test_tangle_data.reset_index(drop=True)
test_tangle_data['ElementId'] = test_tangle_data['ElementId'].astype('int64')

tangle_groups = test_tangle_data.groupby('ElementId')['Position'].apply(list).reset_index()
#==== Concat all csv
import glob
def multi_concat():
    csv_files = glob.glob('*.csv')
    out_df = pd.DataFrame()
    for file in csv_files:
        if "revit_params_KRGN" in file:
            print(file)
            df = pd.read_csv(file)
            
            if out_df.empty:
                out_df = df
            else:
                out_df = pd.concat([out_df, df], sort=False, axis=0)
    
    return out_df

test_revit_data = multi_concat()

"""
по началу я не хотел марджить, и по идее в проде это тоже не понадобится, но что бы сделать 
замеры метрик видимо нужно мерджить, что бы выровнять датасеты. Иначе обьекты и метки будут в разных позициях
что приведет к неправильному расчету метрик
"""



revit_params_KRGN-MK_GP07_AR_01.csv


C:\Users\homer\AppData\Local\Temp\ipykernel_15204\2450841528.py:29: DtypeWarning: Columns (195,220,221,231,234,235,237,240,245,247,253,254,255,258,259,260,262,268,270,271,277,284,286,288,290,314,317,318,319,333,363,364,370,402,403,417,423,424,462,465,477,478,484,509,527,649,673,674,677,678,680,766,767,795,796,797,798,799,800,808,814,815,820,821,822,823,824,825,828,829,830,831,832,833,834,835,836,837,838,864,866,898,899,927,931,979,982,983,984,985,992,997,998,999,1001,1002,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1051,1077,1089,1093,1094,1099,1105,1142,1147,1159,1161,1162,1163,1164,1165,1166,1167,1168,1171,1172,1173,1174,1175,1176,1177,1178,1179,1185,1187,1204,1216,1228,1241,1271,1366,1447,1496,1504,1513,1569,1570,1571,1572,1587,1606,1607,1609,1632,1665,1666,1681,1682,1729,1740,1741,1743,1745,1748,1750,175

revit_params_KRGN-MK_GP07_AR_02.csv


C:\Users\homer\AppData\Local\Temp\ipykernel_15204\2450841528.py:29: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,171,172,173,174,177,182,183,185,186,187,188,190,191,192,193,194,195,197,199,201,202,204,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,2

revit_params_KRGN-MK_GP07_AR_03.csv


...

revit_params_KRGN-MK_GP07_AR_09.csv


C:\Users\homer\AppData\Local\Temp\ipykernel_15204\2450841528.py:29: DtypeWarning: Columns (205,206,216,221,222,224,227,232,234,241,242,243,246,247,248,250,256,258,259,265,272,274,277,279,280,303,306,307,308,309,310,319,321,322,330,331,364,365,371,403,404,419,425,426,428,478,483,484,497,503,527,544,592,593,594,595,596,597,598,599,600,679,705,706,709,710,712,799,804,806,807,808,836,837,838,839,840,841,849,854,855,860,861,862,863,864,865,868,869,870,871,872,873,874,875,876,877,878,904,907,943,944,969,988,1015,1018,1019,1020,1021,1024,1028,1034,1035,1036,1038,1039,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1112,1124,1128,1129,1130,1135,1141,1170,1182,1187,1194,1200,1202,1203,1204,1205,1206,1207,1208,1209,1212,1213,1214,1215,1216,1217,1218,1219,1220,1226,1228,1244,1256,1271,1285,1319,1416,1496,1546,1555,1564,1618,161

revit_params_KRGN-MK_GP07_AR_10.csv


C:\Users\homer\AppData\Local\Temp\ipykernel_15204\2450841528.py:29: DtypeWarning: Columns (184,216,217,227,232,233,235,243,245,251,252,253,256,257,258,260,266,268,269,275,282,284,287,289,290,313,316,317,318,319,320,327,328,330,331,339,340,342,372,379,411,412,427,433,434,436,483,488,489,501,505,514,550,567,617,618,619,620,621,622,623,624,625,705,731,732,735,736,738,828,832,834,835,836,848,865,866,867,868,869,870,878,884,885,890,891,892,893,894,895,898,899,900,901,902,903,904,905,906,907,908,936,938,979,980,1009,1028,1050,1053,1054,1055,1056,1059,1064,1070,1071,1072,1074,1075,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1112,1113,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1151,1166,1169,1170,1171,1176,1182,1223,1228,1235,1241,1243,1244,1245,1246,1247,1248,1249,1250,1253,1254,1255,1256,1257,1258,1259,1260,1261,1267,1269,1286,1300,1315,1329,1365,1463,1543,1593

'\nпо началу я не хотел марджить, и по идее в проде это тоже не понадобится, но что бы сделать \nзамеры метрик видимо нужно мерджить, что бы выровнять датасеты. Иначе обьекты и метки будут в разных позициях\nчто приведет к неправильному расчету метрик\n'

In [11]:
#=== Marge
# tangle_groups = tangle_groups.sample(frac=1, random_state=42).reset_index()
# del tangle_groups['index']
test_marged_df = tangle_groups.merge(test_revit_data, left_on='ElementId', right_on='ElementId')
# test_marged_df = test_marged_df.sample(frac=1, random_state=42).reset_index()

~~Преобразования датафреймов. y - tangle_groups, x - test_revit_data~~

In [12]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

#=== marged_df split and clear
target_cols = ['ElementId', 'Position']
test_ids = test_marged_df['ElementId'].copy()
X = test_marged_df.drop(columns=target_cols, axis=1)
to_drop = ['Марка', 'Граница помещения','Комментарии','Линия привязки','Маркировка типоразмера','ADSK_Этаж', 'Цвет штриховки при низкой детализации']
X.drop(columns=to_drop, inplace=True)
X['Зависимость снизу'] = X['Зависимость снизу'].str.split('(', expand=True)[0]
X['Зависимость сверху'] = X['Зависимость сверху'].str.split('(', expand=True)[0]
X['Уровень'] = X['Уровень'].str.split('(', expand=True)[0]

y = test_marged_df[target_cols]

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_fit = mlb.fit_transform(y['Position'])
y = pd.DataFrame(y_fit, columns=mlb.classes_,index=y['ElementId'])
y = y.add_prefix('Target_')

with open('features.json', 'r') as f:
    features = json.load(f)

#=== Выбор только нужных фичей
X = X[X.columns.intersection(features)]

# constant_cols = X.columns[X.nunique() <= 1]
# X = X.drop(columns=constant_cols)
# print("Константные колонки:", len(list(constant_cols)))

# X.fillna("NAN", inplace=True)

In [20]:
"""
Получилось так, что на тесте больше таргетов, чем на обучении. 
Надо занулить лишние таргеты, что бы занулить вероятности при обучении.
"""
y.columns.__len__() # 31 таргеты на тестовых данных 
y.columns.__len__() # 29 таргеты на обучающих данных 


37

### Убрал MultiOutputClassifie, что бы получить более кнтролируемое качество. 

In [13]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.utils.class_weight import compute_sample_weight
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib
import json

with open('cat_features.json', 'r') as f:
    test_cat_features = json.load(f)

for col in test_cat_features:
    try:
        X[col] = X[col].astype(str)
        """
        Чести параметров что были на обучении может не быть. Если такое произайдет, то скорее
        всего такое параметр не сильно влияет. Но что бы все работало корректно, такие параметры 
        заполняются специальным пропуском. 
        """
        X[col] = X[col].astype('category')
    except KeyError:
        print(f"Параметр '{col}' не найден в данных и будет заполнен как 'missing'")
        X[col] = 'missing'
        # test_cat_features.remove(col)
        continue  # или ничего не делать

catb_model = CatBoostClassifier(
    random_state=42,
    nan_mode='Min',           # или 'Min' тут хз
    max_depth=4,              
    learning_rate=0.5,        # мб стоит изсенить 
    n_estimators=200,         # количество деревьев пока 200 убрал, так как пробую использовать автоостановку
    verbose=100,              # лог каждые 100 итераций, иначе иного инфы
    # od_wait=10,
    scale_pos_weight=None,    # комит, так как не задаю class_weights
    cat_features=cat_features,# база
    eval_metric='F1',         # попробую задалансить f1
    auto_class_weights='Balanced',  # !автомитическая балансировака классов. Имба
)


#=== Загрузка
print("Загрузка CatBoost завершена")
# multi_lgbm = joblib.load('lgbm.pkl')
multi_catb = joblib.load('catb.pkl')

Загрузка CatBoost завершена


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import joblib

lgbm = LGBMClassifier(
    random_state=42,
    num_leaves=10,
    max_depth=5,
    device='gpu',
    use_missing=True, 
    zero_as_missing=False,

    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.1,
    min_child_samples=20,
)

catb_model = CatBoostClassifier(
    random_state=42,
    nan_mode="Min",
    max_depth=5
)

# multi_lgbm = MultiOutputClassifier(lgbm)

with open('cat_features.json', 'r') as f:
    test_cat_features = json.load(f)

for col in test_cat_features:
    try:
        X[col] = X[col].astype(str)
        """
        Чести параметров что были на обучении может не быть. Если такое произайдет, то скорее
        всего такое параметр не сильно влияет. Но что бы все работало корректно, такие параметры 
        заполняются специальным пропуском. 
        """
        X[col] = X[col].astype('category')
    except KeyError:
        print(f"Параметр '{col}' не найден в данных и будет заполнен как 'missing'")
        X[col] = 'missing'
        # test_cat_features.remove(col)
        continue  # или ничего не делать

multi_catb = MultiOutputClassifier(catb_model)

#=== Загрузка
print("Загрузка LightGBM и CatBoost завершена")
# multi_lgbm = joblib.load('lgbm.pkl')
multi_catb = joblib.load('catb.pkl')


Параметр 'Марка' не найден в данных и будет заполнен как 'missing'
Загрузка LightGBM и CatBoost завершена


In [14]:
# Предсказание вероятностей
# y_pred_proba_lgbm = np.array([clf.predict_proba(X)[:,1] for clf in multi_lgbm.estimators_]).T
y_pred_proba_ctb = np.array([clf.predict_proba(X)[:,1] for clf in multi_catb]).T

# усреднение вероятностей
# y_pred_proba_ensemble = (y_pred_proba_lgbm + y_pred_proba_catb) / 2

# отсечения порогом вероятности 
y_pred_ensemble = (y_pred_proba_ctb > 0.5).astype(int)

Логика в том что берем только те таргеты которые были в обучении

In [ ]:
"""
Получилось так, что на тесте больше таргетов, чем на обучении. y_pred_ensemble.shape - (220244, 29)
Надо занулить лишние таргеты, что бы занулить то, что не предсказано, что бы не ломать метрики. но
У меня сейчас супер мало данных. Так что придется проверять только то что есть. 
"""
y.columns.__len__() # - 37 таргеты на тестовых данных 
y_train.columns.__len__() # - 25 таргеты на обучающих данных 

expected_targets = y.columns.tolist()
with open('trained_targets.json', 'r') as f:
    trained_targets = json.load(f)
trained_targets
print(expected_targets)
print(trained_targets)

indexes = []
data_targets = []
for ind, targ in enumerate(trained_targets):
    if targ in expected_targets :
        print(targ)
        indexes.append(ind)
        data_targets.append(targ)

new_pred = y_pred_ensemble[:, indexes]
new_y = y[data_targets]
# new_y
new_y

['Target_004.014.033.001.002', 'Target_004.014.033.001.005', 'Target_004.014.033.001.008', 'Target_004.014.033.001.012', 'Target_004.014.033.001.013', 'Target_004.014.033.001.015', 'Target_004.014.033.001.3', 'Target_004.014.033.002.001', 'Target_004.014.033.002.002', 'Target_004.014.033.002.003', 'Target_004.014.033.002.004', 'Target_004.014.033.002.005', 'Target_004.014.033.004.001', 'Target_004.014.033.004.002', 'Target_004.014.033.004.003', 'Target_004.014.033.004.004', 'Target_004.014.033.004.005', 'Target_004.014.033.004.006', 'Target_004.014.033.006.001.1', 'Target_004.014.033.006.002', 'Target_021.004.003', 'Target_021.005.003.005.073', 'Target_021.005.003.005.074', 'Target_021.005.003.005.075', 'Target_021.005.003.005.082', 'Target_021.015.001.017', 'Target_031.003.002.006', 'Target_031.003.003.001', 'Target_031.003.003.002', 'Target_031.003.003.003', 'Target_031.003.003.004', 'Target_031.003.003.005', 'Target_031.006.003', 'Target_031.006.004', 'Target_031.006.006', 'Target_0

,Target_004.014.033.001.002,Target_004.014.033.001.005,Target_004.014.033.001.008,Target_004.014.033.001.012,Target_004.014.033.001.013,Target_004.014.033.002.001,Target_004.014.033.002.002,Target_004.014.033.002.003,Target_004.014.033.002.004,Target_004.014.033.002.005,Target_004.014.033.004.001,Target_004.014.033.004.002,Target_004.014.033.004.003,Target_004.014.033.004.004,Target_004.014.033.004.005,Target_004.014.033.004.006,Target_004.014.033.006.001.1,Target_021.015.001.017,Target_031.006.003,Target_031.006.006
ElementId,,,,,,,,,,,,,,,,,,,,
1128759,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128759,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128760,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128760,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1128761,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5309498,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6114827,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
6116494,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
# тут сравнения по 2-м датафреймам
from sklearn.metrics import accuracy_score, hamming_loss, f1_score, precision_score, recall_score
#=== Точность по объектам 
exact_match = accuracy_score(new_y,new_pred, normalize=True)

#=== Hamming Accuracy - доля правильно предсказанных меток
hamming_acc = 1 - hamming_loss(new_y, new_pred)

#=== F1, Precision, Recal
f1_macro = f1_score(new_y, new_pred, average='macro')
f1_micro = f1_score(new_y, new_pred, average='micro')

precision_macro = precision_score(new_y, new_pred, average='macro', zero_division=0)
precision_micro = precision_score(new_y, new_pred, average='micro', zero_division=0)

recall_macro = recall_score(new_y, new_pred, average='macro', zero_division=0)
recall_micro = recall_score(new_y, new_pred, average='micro', zero_division=0)

#=== Метрики
print("Результаты CatBoost на тестовых данных:")
print(f"Macro F1 Score:       {f1_macro:.4f}")
print(f"Micro F1 Score:       {f1_micro:.4f}")
print(f"Macro Precision:      {precision_macro:.4f}")
print(f"Micro Precision:      {precision_micro:.4f}")
print(f"Macro Recall:         {recall_macro:.4f}")
print(f"Micro Recall:         {recall_micro:.4f}")

Результаты CatBoost на тестовых данных:
Macro F1 Score:       0.5106
Micro F1 Score:       0.7218
Macro Precision:      0.5085
Micro Precision:      0.7061
Macro Recall:         0.5720
Micro Recall:         0.7381
